# Model Monitoring for Unmanaged Model
<table align="left">
  <td>
    <a href="https://colab.research.google.com/drive/1bRsBBhlNLZYrIe59UqtdTLd91FCtXmn4">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> Run in Colab
    </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://storage.googleapis.com/cmm-public-data/notebooks/Model_Monitoring_for_Unmanaged_Model.ipynb">
       <img src="https://www.gstatic.com/cloud/images/navigation/vertex-ai.svg" alt="Vertex AI logo">Open in Vertex AI Workbench
    </a>
  </td>
</table>

## Step 1: Authentication

Depending on your Jupyter environment, you may have to manually authenticate. Follow the relevant instructions below.

**1. Vertex AI Workbench**
* Do nothing as you are already authenticated.

**2. Colab, run:**

In [ ]:
from google.colab import auth
auth.authenticate_user()


**3. Local JupyterLab instance, uncomment and run:**

In [ ]:
# ! gcloud auth login

**4. Service account or other**
* See how to grant Cloud Storage permissions to your service account at https://cloud.google.com/storage/docs/gsutil/commands/iam#ch-examples.

## Step 2: Installations & Setup
### Install the following packages to execute this notebook.

In [ ]:
! pip3 install --upgrade --quiet \
    google-cloud-aiplatform \
    google-cloud-bigquery \
    pandas-gbq \
    'tensorflow_data_validation[visualization]<2'

# Model Monitoring Experimental SDK
! gsutil cp gs://cmm-public-data/sdk/google_cloud_aiplatform-1.36.dev20231025+centralized.model.monitoring-py2.py3-none-any.whl .
! pip install --quiet google_cloud_aiplatform-1.36.dev20231025+centralized.model.monitoring-py2.py3-none-any.whl

### Restart the kernel (only for Colab)

In [ ]:
# Automatically restart kernel after installs so that your environment can access the new packages
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

### Setup GCP Project ID and Initialize Vertex AI SDK for Python

In [ ]:
import os

PROJECT_ID = "[your-project-id]" # @param {type:"string"}
# set the project id
! gcloud config set project $PROJECT_ID
os.environ["GOOGLE_CLOUD_PROJECT"] = PROJECT_ID

REGION = "us-central1"
! gcloud config set ai/region $REGION

In [ ]:
# Initialize Vertex AI SDK for Python
import google.cloud.aiplatform as aiplatform

aiplatform.init(project=PROJECT_ID, location=REGION)

### Create a Cloud Storage bucket

Create a storage bucket to store artifacts such as datasets.

In [ ]:
# Create a Cloud Storage bucket
BUCKET_URI = f"gs://your-bucket-name-{PROJECT_ID}-unique"  # @param {type:"string"}

**Only if your bucket doesn't already exist**: Run the following cell to create your Cloud Storage bucket.

In [ ]:
! gsutil mb -l {REGION} -p {PROJECT_ID} {BUCKET_URI}

## Step 3: Prepare your production data in BigQuery

**For unmanaged model, we currently only support consuming production prediction data from BiqQuery.**

We require features in separate columns, example BigQuery schema:

<img src="https://services.google.com/fh/files/misc/example_bq_schema.png" width="400" height="300"/>

Note: If you want to setup continous monitoring with a time window, a timestamp column is required.

**For running this tutorial, let's create some fake serving data**

In [ ]:
import pandas as pd
import numpy as np
import random


# Define the number of rows
num_random = 100000

data = {
    'island': [random.choice([0, 1, 2]) for _ in range(num_random)],
    'culmen_length_mm': [random.normalvariate(50, 3) for _ in range(num_random)],
    'culmen_depth_mm': [random.normalvariate(20, 3) for _ in range(num_random)],
    'flipper_length_mm': [random.randint(160, 250) for _ in range(num_random)],
    'body_mass_g': [random.randint(3000, 8000) for _ in range(num_random)],
    'sex': [random.choice([0, 1]) for _ in range(num_random)]
}

# Create a DataFrame from the generated data
df = pd.DataFrame(data)

# Define the time range (start and end dates) in UTC
# now-24h ~ now + 24h
start_date = pd.Timestamp.utcnow() - pd.Timedelta(days=1)
end_date = pd.Timestamp.utcnow() + pd.Timedelta(days=1)

# Generate a list to store the random timestamps
random_timestamps = []

# Generate random timestamps and add them to the list
for _ in range(num_random):
    random_seconds = np.random.randint((end_date - start_date).total_seconds())
    random_timestamp = start_date + pd.Timedelta(seconds=random_seconds)
    # Format the timestamp as a string with microseconds
    formatted_timestamp = random_timestamp.strftime('%Y-%m-%d %H:%M:%S.%f UTC')
    random_timestamps.append(formatted_timestamp)

df['timestamp'] = random_timestamps

df.to_csv('production.csv', index=False)

Create a BigQuery dataset and load the fake data to a table.

In [ ]:
import pandas as pd

TIMESTAMP = pd.Timestamp.utcnow().strftime('%Y%m%d%H%M%S')

FAKE_DATA_BQ_DATASET=f"penguins_production_{TIMESTAMP}"
!bq mk --dataset $PROJECT_ID:$FAKE_DATA_BQ_DATASET

In [ ]:
FAKE_DATA_BQ_TABLE=f"{FAKE_DATA_BQ_DATASET}.data"
!bq load --autodetect --source_format=CSV $FAKE_DATA_BQ_TABLE "production.csv"

Check the serving logging table.

In [ ]:
import pandas as pd

query_string = f"SELECT * FROM `{FAKE_DATA_BQ_TABLE}` ORDER BY timestamp DESC LIMIT 10"
pd.read_gbq(query_string, project_id=PROJECT_ID)

## Step 4: Create a Model Monitor

In [ ]:
from google.cloud.aiplatform.private_preview.centralized_model_monitoring import model_monitor

my_model_monitor = model_monitor.ModelMonitor.create(
    project=PROJECT_ID,
    location=REGION,
    model_name="penguins")
MODEL_MONITOR_RESOURCE_NAME = my_model_monitor.name
print(f"MODEL MONITOR {MODEL_MONITOR_RESOURCE_NAME} created.")

## Step 5: Create an on-demand Model Monitoring Job

In [ ]:
# Copy files to your projects gs bucket to avoid permission issues.
# Ignore any error(s) for bucket already exists.
PUBLIC_TRAINING_DATASET = "gs://cmm-public-data/datasets/penguins/training_100k.csv"
TRAINING_DATASET = f"{BUCKET_URI}/penguins/training_100k.csv"

! gsutil copy $PUBLIC_TRAINING_DATASET $TRAINING_DATASET

In [ ]:
import pandas as pd

EMAIL="[your-email-address]" # @param {type:"string"}

# Skew and drift thresholds.
DEFAULT_THRESHOLD_VALUE = 0.001

SKEW_THRESHOLDS = {
    "culmen_length_mm": DEFAULT_THRESHOLD_VALUE,
    "body_mass_g": 0.002,
}

# Prediction target column name in training dataset.
GROUND_TRUTH = "species"

TIMESTAMP = pd.Timestamp.utcnow().strftime('%Y%m%d%H%M%S')
JOB_DISPLAY_NAME = f"churn_model_monitoring_job_{TIMESTAMP}"

Let's start a monitoring job for the skew detection(training vs serving).
In this example, training data is a csv file from Google Cloud Storage and the serving data is from BigQury. We support two options for connection:

* table_uri: This is what the example shows, it will consume all the features from the table.
* query: Actually it's SQL query, you could select the features you are interested for analysis, be sure to include the timestamp column if you'd like to specify the data window or want the continous monitoring.

In [ ]:
model_monitoring_job=my_model_monitor.run(
    display_name=JOB_DISPLAY_NAME,
    objective_config=model_monitor.spec.ObjectiveSpec(
        baseline=model_monitor.spec.MonitoringInput(
            gcs_uri=TRAINING_DATASET,
            data_format="csv",
            ground_truth_field=GROUND_TRUTH),
        target=model_monitor.spec.MonitoringInput(
            table_uri=f"bq://{PROJECT_ID}.{FAKE_DATA_BQ_TABLE}",
            timestamp_field="timestamp"),
        feature_distribution_skew=model_monitor.spec.SkewSpec(
            default_threshold=DEFAULT_THRESHOLD_VALUE,
            feature_thresholds=SKEW_THRESHOLDS,
            # The data window of the serving data is "2h", indicating the selection of '2-hour' data windows before the current time for analysis.
            window="2h")
    ),
    notification_config=model_monitor.spec.NotificationSpec(
        user_emails=[EMAIL],
    ),
    output_config=model_monitor.spec.OutputSpec(
        gcs_base_dir=BUCKET_URI
    )
)

CMM_JOB_RESOURCE_NAME = model_monitoring_job.name
print(f"Model Monitoring Job {CMM_JOB_RESOURCE_NAME} created.")

## Step 6: Wait for the Model Monitoring Job to finish and verify the result

### Check email

#### Here's a sample create job email...

<img src="https://services.google.com/fh/files/misc/ad_hoc_email.png" />

#### If there is any anomaly detected, you will receive an email like

<img src="https://services.google.com/fh/files/misc/ad_hoc_anomalies_email.png" />

### Check GCP Console

#### Check the "Monitor" tab under "Vertex AI"

<img src="https://storage.googleapis.com/cmm-public-data/images/unmanaged_job_console.gif" />

### Verify Output GCS bucket after job is finished

In [ ]:
my_model_monitor.show_skew_stats(model_monitoring_job_name=CMM_JOB_RESOURCE_NAME)

## Step 7: Schedule Continous Model Monitoring

If you are interested at trying continous model monitoring, please following the example below to create a schedule. You could create multiple schedules for your monitor.

This example is to run the model monitoring job every 1 hour at 00:00, 01:00 ... Everytime "1h" window data will be analyzed. Let's say a job is scheduled to run at 6:00am, then 5:00 am ~ 6:00 am data will be consumed for analysis.

In [ ]:
CRON="0 * * * *" # @param {type:"string"} Every 1 hour at :00, for example 1:00, 2:00..
SCHEDULE_DISPLAY_NAME="penguins-continous-skew-detection"
DATA_WINDOW="1h"

In [ ]:
model_monitoring_schedule=my_model_monitor.create_schedule(
    display_name=SCHEDULE_DISPLAY_NAME,
    cron=CRON,
    objective_config=model_monitor.spec.ObjectiveSpec(
        baseline=model_monitor.spec.MonitoringInput(
            gcs_uri=TRAINING_DATASET,
            data_format="csv",
            ground_truth_field=GROUND_TRUTH),
        target=model_monitor.spec.MonitoringInput(
            table_uri=f"bq://{PROJECT_ID}.{FAKE_DATA_BQ_TABLE}",
            timestamp_field="timestamp"),
        feature_distribution_skew=model_monitor.spec.SkewSpec(
            default_threshold=DEFAULT_THRESHOLD_VALUE,
            feature_thresholds=SKEW_THRESHOLDS,
            window=DATA_WINDOW),

    ),
    notification_config=model_monitor.spec.NotificationSpec(
        user_emails=[EMAIL],
    ),
    output_config=model_monitor.spec.OutputSpec(
        gcs_base_dir=BUCKET_URI
    )
)

SCHEDULE_RESOURCE_NAME = model_monitoring_schedule.name
print(f"Schedule {SCHEDULE_RESOURCE_NAME} created.")

You could check out your schedules in Console
<img src="https://storage.googleapis.com/cmm-public-data/images/unmanaged_continous.gif" />

## Step 8: Clean Up (after job finished)

In [ ]:
from google.cloud import bigquery

# When no jobs are running, delete the schedule and all the jobs.
my_model_monitor.delete_schedule(SCHEDULE_RESOURCE_NAME)
my_model_monitor.delete_all_model_monitoring_jobs()
my_model_monitor.delete()

# Delete BQ logging table
bqclient = bigquery.Client(project=PROJECT_ID)
# Delete the dataset (including all tables)
bqclient.delete_dataset(FAKE_DATA_BQ_DATASET, delete_contents=True, not_found_ok=True)